In [ ]:
import holoviews as hv
import hvplot
import hvplot.xarray
import xarray as xr
import numpy as np

from snobedo.lib.dask_utils import start_cluster, client_ip_and_port
from common import SHARED_STORE, HV_PLOT_OPTS

hv.extension('bokeh')

In [ ]:
client = start_cluster(6, 24)
client_ip_and_port(client)

In [ ]:
HV_PLOT_OPTS['ylabel'] = 'Net Solar W/m^2'

year = 2021
water_year = f'wy{year}'

In [ ]:
smrf_eb = xr.open_mfdataset(
    f'{SHARED_STORE}/erw_isnobal/GMD/{water_year}/erw/run*/smrf_energy*.nc',
    parallel=True,
)

# Net Solar

In [ ]:
HRRR_solar = xr.open_mfdataset(
    f'{SHARED_STORE}/erw_isnobal/{water_year}/erw_hrrr_solar_modis/run*/net_solar.nc',
    parallel=True,
)

In [ ]:
HRRR_solar_cubic = xr.open_mfdataset(
    f'{SHARED_STORE}/erw_isnobal/{water_year}/erw_hrrr_solar_modis_cubic/run*/net_solar.nc',
    parallel=True,
)

## Daily mean area 

In [ ]:
SMRF_mean_area_solar = smrf_eb.net_solar.mean(['x', 'y']).compute()

HRRR_mean_area_solar = HRRR_solar.net_solar.mean(['x', 'y']).compute()

HRRR_mean_area_solar_cubic = HRRR_solar_cubic.net_solar.mean(['x', 'y']).compute()

### SMRF vs. HRRR 

In [ ]:
HRRR_mean_area_solar.hvplot(label='Net Solar HRRRR MODIS', color='goldenrod').opts(**HV_PLOT_OPTS) * \
    HRRR_mean_area_solar_cubic.hvplot(label='Net Solar HRRRR MODIS CUBIC', color='orangered') * \
    SMRF_mean_area_solar.hvplot(label='Net Solar SMRF', color='dodgerblue')

### Rolling window (daily) 

In [ ]:
HRRR_mean_area_solar.rolling(time=24, center=True).mean().hvplot(label='HRRR', color='goldenrod').opts(**HV_PLOT_OPTS) * \
    SMRF_mean_area_solar.rolling(time=24, center=True).mean().hvplot(label='SMRF', color='dodgerblue')

In [ ]:
# HRRR_solar.resample(time='1D').mean().net_solar.hvplot.quadmesh(x='x', y='y', widget_location='bottom', cmap='viridis', **HV_PLOT_OPTS)

# Cloud factor

In [ ]:
HRRR_cloud = xr.open_mfdataset(
    f'{SHARED_STORE}/erw_isnobal/{water_year}/erw_hrrr_solar/run*/cloud_factor.nc',
    parallel=True,
)

In [ ]:
smrf_out = xr.open_mfdataset(
    f'{SHARED_STORE}/erw_isnobal/GMD/{water_year}/erw/run*/smrf_20*.nc',
    parallel=True,
)

In [ ]:
HV_PLOT_OPTS = dict(
    width=1200, 
    height=600,
    ylabel='1 -> Clear sky; 0 -> Full cloud cover',
)

## Daily mean area

In [ ]:
SMRF_mean_area_solar = smrf_out.cloud_factor.mean(['x', 'y']).compute()

In [ ]:
HRRR_mean_area_solar = HRRR_cloud.TCDC.mean(['x', 'y']).compute()

### SMRF vs. HRRR 

In [ ]:
HRRR_mean_area_solar.hvplot(label='Cloud Factor HRRRR', color='goldenrod').opts(**HV_PLOT_OPTS) * \
    SMRF_mean_area_solar.hvplot(label='Cloud Factor SMRF', color='dodgerblue')

### Rolling window (daily) 

In [ ]:
HRRR_mean_area_solar.rolling(time=24, center=True).mean().hvplot(label='HRRR', color='goldenrod').opts(**HV_PLOT_OPTS) * \
    SMRF_mean_area_solar.rolling(time=24, center=True).mean().hvplot(label='SMRF', color='dodgerblue')

# Thermal

In [ ]:
HRRR_therm = xr.open_mfdataset(
    f'{SHARED_STORE}/erw_isnobal/{water_year}/erw_hrrr_solar/run*/smrf_20*.nc',
    parallel=True,
)

## Daily mean area

In [ ]:
SMRF_mean_area_solar = smrf_out.thermal.mean(['x', 'y']).compute()

In [ ]:
HRRR_mean_area_solar = HRRR_therm.thermal.mean(['x', 'y']).compute()

### SMRF vs. HRRR 

In [ ]:
HRRR_mean_area_solar.hvplot(label='Thermal HRRRR', color='goldenrod').opts(**HV_PLOT_OPTS) * \
    SMRF_mean_area_solar.hvplot(label='Thermal SMRF', color='dodgerblue')

### Rolling window (daily) 

In [ ]:
HRRR_mean_area_solar.rolling(time=24, center=True).mean().hvplot(label='HRRR', color='goldenrod').opts(**HV_PLOT_OPTS) * \
    SMRF_mean_area_solar.rolling(time=24, center=True).mean().hvplot(label='SMRF', color='dodgerblue')

### SMRF net solar components (IR/VIS)

In [ ]:
SMRF_mean_area_ir = smrf_solar.ir_in.mean(['x', 'y']).compute()
SMRF_mean_area_vis = smrf_solar.vis_in.mean(['x', 'y']).compute() 

In [ ]:
SMRF has a max of 800, configured in the .ini file

In [ ]:
SMRF_mean_area = (SMRF_mean_area_ir + SMRF_mean_area_vis).compute()
SMRF_mean_area[SMRF_mean_area > 800] = 800

In [ ]:
(hv.Curve(SMRF_mean_area, label='SUM SMRF').opts(**HV_PLOT_OPTS) * \
    SMRF_mean_area_ir.hvplot(label='IR') * \
    SMRF_mean_area_vis.hvplot(label='VIS') * \
    hv.HLine(800).opts(color='purple')).opts(legend_position='bottom_right')